In [1]:
import torch
from glob import glob
from ast import literal_eval

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List
from IPython.core.display import display, HTML
from sklearn.manifold import TSNE
from tqdm.notebook import tqdm

In [2]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)

plt.style.use('dark_background')

display(HTML("<style>"
    + "#notebook { padding-top:0px; } " ""
    + ".container { width:100%; } "
    + ".end_space { min-height:0px; } "
    + "</style>"))

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda


In [3]:
def get_name_exp(exp: dict):
    interesting_props = ['buffer_size', 'con_weight']
    props = {}
    for prop in interesting_props:
        if prop in exp:
            props[prop] = exp[prop]
    if len(props) == 0: props = ''
    return f'{exp["model"]} {props}'

def print_exp_info(exp: dict):
    print(f'model: {get_name_exp(exp)}')
    print({k: exp[k] for k in exp if k not in ['latents']})

In [4]:
def flatten_list(l: List[List[any]]) -> List[any]:
    return [item for line in l for item in line]

In [5]:
# logs_path = 'C:\\Users\\emace\\AImageLab\\SRV-Continual\\results\\canomaly\\logs'
logs_path = '/nas/softechict-nas-2/efrascaroli/mammoth-data/logs'
# logs_path = '/nas/softechict-nas-1/rbenaglia/canomaly-data/logs'
# logs_path = '../storage/logs'

In [6]:
exp_list = []
environments = []
for log_file in glob(logs_path + '/**/*ace*/*res*.pyd', recursive=True):
    print(log_file)
    with open(log_file, 'r') as f:
        props_list = [prop for prop in log_file.replace(logs_path, '').split('/')[1:]]
        print(f'Env: {props_list}')
        props = props_list
        exps = []
        for i, line in enumerate(f.readlines()):
            try:
                exps.append(literal_eval(line.replace('nan', 'None')))
            except:
                print(f'Unparsed line {i}:\n\t{exps[:-1]}\n-->\t{line}')

        environments.append({'env': props, 'exps': exps})

        exp_list.extend(exps)

/nas/softechict-nas-2/efrascaroli/mammoth-data/logs/seq-cifar10/er_ace_con/results.pyd
Env: ['seq-cifar10', 'er_ace_con', 'results.pyd']
/nas/softechict-nas-2/efrascaroli/mammoth-data/logs/seq-cifar10/er_ace_con/res1.pyd
Env: ['seq-cifar10', 'er_ace_con', 'res1.pyd']
/nas/softechict-nas-2/efrascaroli/mammoth-data/logs/seq-cifar100-10x10/er_ace_con/res1.pyd
Env: ['seq-cifar100-10x10', 'er_ace_con', 'res1.pyd']


In [7]:
print(f'Total exps: {len(exp_list)}')
# exp_list[0]['results']
environments[1]['env']

Total exps: 50


['seq-cifar10', 'er_ace_con', 'res1.pyd']

In [8]:
discriminant = 'con_weight'
important_task = [2, 3, 4]
results_dict = {}
for idx, exp in enumerate(environments[0]['exps']):
    dis = exp[discriminant]
    if dis not in results_dict:
        results_dict[dis] = {'n_exps': 0, 'exp_indexes': []}
        for i in important_task:
            results_dict[dis][f'CIL-{i}'] = []
            results_dict[dis][f'TIL-{i}'] = []
            results_dict[dis][f'ConE-{i}'] = []
    results_dict[dis]['n_exps'] += 1
    results_dict[dis]['exp_indexes'].append(idx)
    for i in important_task:
        results_dict[dis][f'CIL-{i}'].append(exp['results'][i-1]['Class-IL mean'])
        results_dict[dis][f'TIL-{i}'].append(exp['results'][i-1]['Task-IL mean'])
        results_dict[dis][f'ConE-{i}'].append(exp['results'][i-1]['Con-Error'])

In [9]:
df = pd.DataFrame(index=[f'Task {i}' for i in important_task],
                  columns=['Class-IL', 'Task-IL', 'Con-Error'])
for dis in results_dict:
    res = results_dict[dis]
    print(f'con weight: {dis} | exps({res["n_exps"]}): {res["exp_indexes"]}\n')
    for i in important_task:
        df.loc[f'Task {i}', 'Class-IL']     = f'{np.mean(res[f"CIL-{i}"]):.2f} ± {np.std(res[f"CIL-{i}"]):.2f}'
        df.loc[f'Task {i}', 'Task-IL']      = f'{np.mean(res[f"TIL-{i}"]):.2f} ± {np.std(res[f"TIL-{i}"]):.2f}'
        df.loc[f'Task {i}', 'Con-Error']    = f'{np.mean(res[f"ConE-{i}"]):.2f} ± {np.std(res[f"ConE-{i}"]):.2f}'
    display(df)

con weight: 0.1 | exps(5): [0, 7, 11, 16, 18]



,Class-IL,Task-IL,Con-Error
Task 2,58.10 ± 4.70,73.22 ± 4.19,24.26 ± 1.00
Task 3,47.71 ± 3.33,76.72 ± 3.34,23.16 ± 0.43
Task 4,41.77 ± 3.39,77.02 ± 2.31,21.47 ± 0.67


con weight: 0.001 | exps(5): [1, 4, 8, 12, 15]



,Class-IL,Task-IL,Con-Error
Task 2,79.90 ± 2.82,91.24 ± 0.98,23.96 ± 0.86
Task 3,64.96 ± 2.70,90.24 ± 0.86,22.56 ± 0.78
Task 4,60.64 ± 3.47,89.52 ± 1.66,21.29 ± 0.74


con weight: 0.01 | exps(5): [2, 5, 9, 13, 17]



,Class-IL,Task-IL,Con-Error
Task 2,80.25 ± 1.51,91.02 ± 0.75,22.78 ± 1.85
Task 3,65.81 ± 2.81,90.42 ± 1.18,21.67 ± 0.61
Task 4,61.70 ± 2.01,90.90 ± 0.67,20.75 ± 0.56


con weight: 0.0 | exps(5): [3, 6, 10, 14, 19]



,Class-IL,Task-IL,Con-Error
Task 2,82.32 ± 1.38,91.92 ± 0.73,26.63 ± 0.95
Task 3,66.85 ± 1.95,90.08 ± 0.91,25.58 ± 0.99
Task 4,61.18 ± 1.39,90.22 ± 0.82,23.91 ± 1.21


In [15]:
discriminant = 'con_weight'
important_task = [2]
divide_accs = [1, 2]
results_dict = {}
for idx, exp in enumerate(environments[2]['exps']):
    dis = exp[discriminant]
    if dis not in results_dict:
        results_dict[dis] = {'n_exps': 0, 'exp_indexes': [], 'Task-1': [], }
        for i in divide_accs:
            results_dict[dis][f'CIL-{i}'] = []
            results_dict[dis][f'TIL-{i}'] = []
            results_dict[dis][f'ConE-{i}'] = []
    results_dict[dis]['n_exps'] += 1
    results_dict[dis]['exp_indexes'].append(idx)
    results_dict[dis]['Task-1'].append(exp['results'][0]['Class-IL'][0])
    for i in important_task:
        for j in divide_accs:
            results_dict[dis][f'CIL-{j}'].append(exp['results'][i-1]['Class-IL'][j-1])
            results_dict[dis][f'TIL-{j}'].append(exp['results'][i-1]['Task-IL'][j-1])
            results_dict[dis][f'ConE-{j}'].append(exp['results'][i-1]['Con-Error'])

In [16]:
df = pd.DataFrame(index=[f'Task {i}' for i in divide_accs],
                  columns=['Class-IL', 'Task-IL', 'Con-Error'])
for dis in results_dict:
    res = results_dict[dis]
    print(f'con weight: {dis} | exps({res["n_exps"]}): {res["exp_indexes"]}\n'
          f'Task-1: {np.mean(res[f"Task-1"]):.2f} ± {np.std(res[f"Task-1"]):.2f}')
    for i in divide_accs:
        df.loc[f'Task {i}', 'Class-IL']     = f'{np.mean(res[f"CIL-{i}"]):.2f} ± {np.std(res[f"CIL-{i}"]):.2f}'
        df.loc[f'Task {i}', 'Task-IL']      = f'{np.mean(res[f"TIL-{i}"]):.2f} ± {np.std(res[f"TIL-{i}"]):.2f}'
        df.loc[f'Task {i}', 'Con-Error']    = f'{np.mean(res[f"ConE-{i}"]):.2f} ± {np.std(res[f"ConE-{i}"]):.2f}'
    display(df)

con weight: 0.0 | exps(6): [0, 1, 6, 7, 10, 11]
Task-1: 76.97 ± 10.08


,Class-IL,Task-IL,Con-Error
Task 1,50.53 ± 3.11,67.98 ± 3.20,26.04 ± 1.40
Task 2,70.90 ± 1.38,84.53 ± 1.53,26.04 ± 1.40


con weight: 0.001 | exps(5): [2, 4, 8, 12, 15]
Task-1: 80.96 ± 3.42


,Class-IL,Task-IL,Con-Error
Task 1,50.68 ± 6.49,69.70 ± 2.09,17.75 ± 0.92
Task 2,68.76 ± 4.33,83.86 ± 2.83,17.75 ± 0.92


con weight: 0.01 | exps(5): [3, 5, 9, 13, 14]
Task-1: 74.18 ± 8.31


,Class-IL,Task-IL,Con-Error
Task 1,53.60 ± 2.64,70.66 ± 3.05,16.63 ± 1.74
Task 2,68.78 ± 4.55,84.12 ± 1.27,16.63 ± 1.74


dict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer2.0.conv1.weight', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.bn1.num_batches_tracked', 'layer2.0.conv2.weight', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', 'l